# Motif Mark Test Notebook

In [ ]:
### Make dictionary of IUPAC ambiguity codes
iupac

In [11]:
import cairocffi as cairo
import math

ModuleNotFoundError: No module named 'cairocffi'

## `Pycairo` Example

In [ ]:
width, height = 800, 500

#create the coordinates to display your graphic, desginate output
surface = cairo.SVGSurface("example.svg",width, height)
#create the coordinates you will be drawing on (like a transparency) - you can create a transformation matrix
context = cairo.Context(surface)
#context.scale(width,height) #will set your drawing surface to a 0.0-1.0 scale

#Need to tell cairo where to put the brush, the color and width, and the shape you want it to draw
#draw a line
context.set_line_width(1)
context.move_to(50,25)        #(x,y)
context.line_to(450,25)
context.stroke()

#draw a rectangle
context.rectangle(100,100,150,350)        #(x0,y0,x1,y1)
context.fill()

context.move_to(25,250)
context.set_source_rgb(.25,.5,.5)
context.set_line_width(5)
context.curve_to(100,400,400,100,750,250)
context.stroke()

context.set_source_rgb(0,.6,.8)
context.arc(500, 300, 50, 0, 2*math.pi)
context.fill_preserve()            #draws filled circle, but preserves for later drawing
context.set_source_rgb(.3,.3,.3)
context.stroke()

#write out drawing
surface.finish()

In [ ]:
WIDTH, HEIGHT = 256, 256

surface = cairo.ImageSurface (cairo.FORMAT_ARGB32, WIDTH, HEIGHT)
ctx = cairo.Context (surface)

ctx.scale (WIDTH, HEIGHT) # Normalizing the canvas

pat = cairo.LinearGradient (0.0, 0.0, 0.0, 1.0)
pat.add_color_stop_rgba (1, 0.7, 0, 0, 0.5) # First stop, 50% opacity
pat.add_color_stop_rgba (0, 0.9, 0.7, 0.2, 1) # Last stop, 100% opacity

ctx.rectangle (0, 0, 1, 1) # Rectangle(x0, y0, x1, y1)
ctx.set_source (pat)
ctx.fill ()

ctx.translate (0.1, 0.1) # Changing the current transformation matrix

ctx.move_to (0, 0)
# Arc(cx, cy, radius, start_angle, stop_angle)
ctx.arc (0.2, 0.1, 0.1, -math.pi/2, 0)
ctx.line_to (0.5, 0.1) # Line to (x,y)
# Curve(x1, y1, x2, y2, x3, y3)
ctx.curve_to (0.5, 0.2, 0.5, 0.4, 0.2, 0.8)
ctx.close_path ()

ctx.set_source_rgb (0.3, 0.2, 0.5) # Solid color
ctx.set_line_width (0.02)
ctx.stroke ()

surface.write_to_png ("example2.png") # Output to PNG